In [ ]:
import deepdish
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import importlib
import magni
import numpy as np
import pandas as pd
from pathlib import Path
import panel as pn
pn.extension()
import xarray as xr

# LOCAL IMPORT
import run_magni

show_axes = opts.Image(xaxis=True, yaxis=True, 
                       xticks=5, yticks=5)

In [2]:
# LOAD IN DATA

stm = deepdish.io.load('data/maksymovych_STM_data.h5')

The purpose of this notebook is to simulate compressive sensing reconstruction as a function of sampling density and added noise. The `run_magni` methods, which are built on top of `magni`, are utilized to accomplish this.

# Performing a single simulation

Simulations can be performed one at a time, as shown below.

In [6]:
import importlib
import run_magni
importlib.reload(run_magni)
importlib.reload(run_magni)
from run_magni import Magni_test as mtest

In [7]:
%%time

from run_magni import Magni_test as mtest

# pick base image (choices are 'tcnq', 'c60', and 'largeFeSe')
name = 'tcnq'
img = stm[name]

# instantiate mtest class to prepare to run simulation
m = mtest(img, name, center=True, stretch=True)

# set noise arguments
noise_args = {'type':'colored', 'beta':1, 'level':0.1}

# run simulation while inputting sampling density, pattern, and other arguments
# see run_magni for more info on arguments
m.run('it', sr_in=0.05, match_n=False, pattern='liss', iterations=1, noise_args=noise_args)

m.plot_results()

C:\Users\Brian\projects\ORNL\cs\CS STM
Wall time: 2.17 s


Row
    [0] HoloViews(Layout)
    [1] DataFrame(DataFrame)

# Looping over parameter space

In [ ]:
%%time
def run_noise_phase_transition(img, name, beta, pattern, noise_levels, sampling_ratios, folder, iterations=300, weights='magni', save=False):

    import pathlib
    from datetime import datetime

    temp_dir = pathlib.Path(f'D:/reconstructions/{folder}')
    if not temp_dir.exists():
        temp_dir.mkdir()
        
    i = 0 
    for noise_level in noise_levels:
        for sr_in in sampling_ratios:
            i += 1
            if i%100 == 0:
                print(noise_level, sr_in)

            # run
            noise_args = {'type':'colored', 'beta':beta, 'level':noise_level}
            m = mtest(img, name, center=True, stretch=True)
            m.run('it', sr_in=sr_in, iterations=iterations, noise_args=noise_args, pattern=pattern, weights=weights)
            ds = m.get_ds(ret_imgs=True)
            
            if save == True:
                # prepare fname
                timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
                fmtd_nl = '{:.2f}'.format(noise_level).split('.')[1]
                fmtd_sr = '{:.2f}'.format(sr_in).split('.')[1]
                fname = temp_dir / f'{m.name}_{m.alg}_sr{fmtd_sr}_nl{fmtd_nl}_{timestamp}.nc' 
                # save
                ds.to_netcdf(fname)
                print(f'saved: {fname}')


# specify parameter space
sampling_ratios = list(np.round(np.arange(0.22,.5+.01,.02), 2)) # extend past original range
noise_levels = list(np.round(np.arange(0.1,1+.1,.1), 2))

# specify additional parameters
beta = 1
iters = 100
weights= 'magni'


for pattern in ['line', 'liss']:
    folder = f'{pattern}_beta{beta}_iters{iters}'
    for k,v in stm.items():
        if True:
            run_noise_phase_transition(v, k, beta, pattern, noise_levels, sampling_ratios,\
                                       folder, iterations=iters, save=True)

# Varying sigma of gaussian weights and threshold

In [ ]:
%%time
def run_phase_transition(img, name, var_dct, pattern, folder, sr_in=0.2, iterations=300, save=False):

    import pathlib
    from datetime import datetime
    from run_magni import Magni_test as mtest

    # set or make dest
    temp_dir = pathlib.Path(f'D:/reconstructions/{folder}')
    if not temp_dir.exists():
        temp_dir.mkdir()

    def check_if_exists(val1, val2, name, folder):
        for p in rec_paths(name, folder):
            if all(check_val(p, t, v) for t, v in [('sig',val1),('thresh',val2)]):
                return True
        return False

    i = 0 
    for val1 in var_dct['dim1']['values']:
        for val2 in var_dct['dim2']['values']:
            fmtd_v1 = var_dct['dim1']['abbr'] + float2str(val1)
            fmtd_v2 = var_dct['dim2']['abbr'] + float2str(val2)
            if check_if_exists(val1, val2, name, folder):
                print(f'{fmtd_v1}, {fmtd_v2}: NOT RUN SINCE COMPLETED PREVIOUSLY')
            else:
                weight_params = {'a': 0.0015, 'd': 0, 'r1': 6.6, 'r2': 1.3, 'sig': val1}
                m = mtest(img, name, center=True, stretch=True)
                m.run('it', sr_in=sr_in, iterations=iterations, pattern=pattern, weights=weight_params, threshold=val2)
                ds = m.get_ds(ret_imgs=True)

                if save == True:
                    # prepare fname
                    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
                    fname = temp_dir / f'{m.name}_{m.alg}_{fmtd_v1}_{fmtd_v2}_{timestamp}.nc' 
                    # save
                    ds.to_netcdf(fname)
                    print(f'saved: {fname}')
                        
                    ### THIS SECTION SAVES THE QUALITY METRICS INTO A SEPARATE FILE ###
                    # open df
                    pt = f'pt/rqm_{name}_{folder}.csv'
                    df = pd.read_csv(pt)
                    #save df
                    add = {'sig':val1, 'thresh':val2}
                    cols = ['ssim', 'dssim', 'mse', 'dmse']
                    add.update({c:ds[c].values.flatten()[0] for c in cols})
                    df = df.append(add, ignore_index=True)
                    df.to_csv(pt, index=False)
    

sigmas = np.round(np.arange(0.1,1.1,0.1),2)
thresholds = np.round(np.linspace(.001, .05, 50), 3)
var_dct = {'dim1':{'full_nm':'sigma','abbr':'sig', 'values':sigmas},
            'dim2':{'full_nm':'threshold','abbr':'thresh', 'values':thresholds}}
iters = 100
sr = 0.2
sr_fmt = '{:.2f}'.format(0.2).split('.')[1]


for pattern in ['line', 'liss']:#, 'liss']:
    for k,v in stm.items():
        if k !='largeFeSe':
            folder = f'{pattern}_iters{iters}_sr{sr_fmt}'
            run_phase_transition(v, k, var_dct, pattern,\
                                       folder, sr_in=0.2, iterations=iters, save=True)